<a href="https://colab.research.google.com/github/himankgalundia/AI-chess-bot-/blob/main/ChessPrototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install patool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 3.0 MB/s eta 0:00:00


In [2]:
!pip3 install python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=579912e187d01a54478bad5be790f649a4375543b235b30ee07a20f579d39116
  Stored in directory: /root/.cache/pip/wheels/f0/3f/76/8783033e8524d407e1bebaf72fdd3f3eba27e0c030e92bbd87
Successfully built chess


In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
import patoolib
import chess
from sklearn.model_selection import train_test_split

board = chess.Board()
print(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [4]:
example_fen = 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1'

In [5]:
def bitMap(fen):
  pieces = [chess.PAWN, chess.BISHOP, chess.KNIGHT, chess.ROOK, chess.QUEEN, chess.KING]
  result = []
  for turn in [chess.WHITE, chess.BLACK]:
    for piece in pieces:
      board = chess.Board(fen)
      piecesArray = board.pieces(piece, turn)
      arr = np.array(piecesArray.tolist())
      arr = np.reshape(arr, (8, 8)).astype(int)
      result.append(arr)
  result = np.transpose(result, axes=(1, 2, 0)) #magic
  return result

In [6]:
data = pd.read_csv('chessData.csv')

In [7]:
data.head()

,FEN,Evaluation
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,-10
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,+56
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,-9
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,+52
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...,-26


TIME TO GET TRAINING DATA

In [8]:
amountToTrainOn = 100
X = np.zeros((amountToTrainOn, 8, 8, 12))
Y = np.zeros((amountToTrainOn,))
for i in range(amountToTrainOn):
  fen = data.iloc[i]['FEN']
  eval = data.iloc[i]['Evaluation']
  if '#' in eval:
    eval = eval[1:]
  bitmap = bitMap(fen)
  X[i] = bitmap
  Y[i] = int(eval)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [12]:
input = tf.keras.layers.Input((8, 8, 12))
x1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu')(input)
x2 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x1)
x3 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(x2)
x4 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(x3)
x5 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(x4)
f = tf.keras.layers.Flatten()(x5)
d1 = tf.keras.layers.Dense(64, activation='relu')(f)
d2 = tf.keras.layers.Dense(32, activation='relu')(d1)
d3 = tf.keras.layers.Dense(1)(d2)
model = tf.keras.models.Model(inputs=input, outputs=d3)

In [13]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 8, 8, 12)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 8, 8, 64)            │           6,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 8, 8, 32)            │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 8, 8, 16)            │           4,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 8, 8, 16)            │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 8, 8, 16)            │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │          65,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 102,417 (400.07 KB)

 Trainable params: 102,417 (400.07 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(loss='mse',
              optimizer='adam',
              metrics=['mse'])
model.fit(x_train,y_train, epochs = 100, batch_size=128, validation_data = (x_test, y_test))

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 60408.3242 - mse: 60408.3242 - val_loss: 84814.5469 - val_mse: 84814.5469
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 60396.7617 - mse: 60396.7617 - val_loss: 84794.1250 - val_mse: 84794.1250
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - loss: 60380.6250 - mse: 60380.6250 - val_loss: 84759.7500 - val_mse: 84759.7500
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step - loss: 60353.7305 - mse: 60353.7305 - val_loss: 84699.7969 - val_mse: 84699.7969
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step - loss: 60307.1758 - mse: 60307.1758 - val_loss: 84599.9375 - val_mse: 84599.9375
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - loss: 60228.6133 - mse: 60228.6133 - val_loss: 84436.1250 - val_mse: 84436.1250
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - loss: 60099.8984 - mse: 60099.8984 - val_loss: 84174.9062 - val_mse: 84174.9062
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - loss: 59895.0742 - 

In [15]:
predictions = model.predict(x_test)
for i in range(len(predictions)):
  print('Prediction: ' + str(predictions[i]) + ', Actual: ' + str(y_test[i]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
Prediction: [62.225014], Actual: 34.0
Prediction: [347.69162], Actual: 569.0
Prediction: [261.72855], Actual: 169.0
Prediction: [8.51142], Actual: -40.0
Prediction: [0.47137046], Actual: 6.0
Prediction: [62.573975], Actual: 7.0
Prediction: [0.33340666], Actual: 0.0
Prediction: [0.48211113], Actual: 0.0
Prediction: [364.09897], Actual: 357.0
Prediction: [1.6444842], Actual: -16.0
Prediction: [0.00403704], Actual: 0.0
Prediction: [182.78497], Actual: 101.0
Prediction: [121.76024], Actual: 456.0
Prediction: [171.69254], Actual: 201.0
Prediction: [60.16896], Actual: 153.0
Prediction: [914.7765], Actual: 929.0
Prediction: [106.663315], Actual: 39.0
Prediction: [16.852058], Actual: -26.0
Prediction: [343.6833], Actual: 253.0
Prediction: [2.0253522], Actual: 50.0


In [16]:
def findMove(fen, turn='white'):
  board = chess.Board(fen)
  turn = 'white'
  if turn == 'white':
    bestMove = ''
    bestEval = -1000000
    newBoard = ''
    for move in board.legal_moves:
      boardCopy = board.copy()
      boardCopy.push(move)
      bitmap = bitMap(boardCopy.fen())
      bitmap = np.reshape(bitmap, (1, 8, 8, 12))
      eval = model.predict(bitmap, verbose=False)
      if eval > bestEval:
        bestEval = eval
        bestMove = move
        newBoard = boardCopy
  else:
    bestMove = ''
    bestEval = 1000000
    newBoard = ''
    for move in board.legal_moves:
      boardCopy = board.copy()
      boardCopy.push(move)
      bitmap = bitMap(boardCopy.fen())
      bitmap = np.reshape(bitmap, (1, 8, 8, 12))
      eval = model.predict(bitmap, verbose=False)
      if eval < bestEval:
        bestEval = eval
        bestMove = move
        newBoard = boardCopy
  return bestMove

In [18]:
startBoard = chess.Board('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1')
print(startBoard)
userinput = input('Enter your first move: ')
while userinput != 'end':
  move = chess.Move.from_uci(userinput)
  startBoard.push(move)
  botMove = findMove(startBoard.fen(), turn = 'black')
  startBoard.push(botMove)
  print()
  print()
  print(startBoard)
  userinput = input('Enter your next move: ')

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


TypeError: 'KerasTensor' object is not callable